El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## 1. Preparación de datos

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score , root_mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import re


In [2]:
# %pip install -U --user scikit-learn

In [3]:
# df = pd.read_csv('/datasets/car_data.csv')

Importacion del dataframe

In [4]:
df = pd.read_csv('D:/Tripleten/datasets/car_data.csv')

Visualización general del dataframe

In [5]:
df.sample(4)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
182207,12/03/2016 15:54,7699,wagon,2007,manual,0,NaN,150000,6,gasoline,volkswagen,NaN,12/03/2016 00:00,0,52353,07/04/2016 04:44
205241,02/04/2016 10:48,4500,small,2008,manual,0,fiesta,100000,0,NaN,ford,NaN,02/04/2016 00:00,0,32602,02/04/2016 10:48
100842,26/03/2016 21:50,950,small,1999,manual,60,punto,150000,10,petrol,fiat,NaN,26/03/2016 00:00,0,79379,06/04/2016 14:15
221283,06/03/2016 15:38,150,wagon,1997,manual,100,vectra,150000,7,petrol,opel,NaN,06/03/2016 00:00,0,96145,06/04/2016 20:45


El objetivo del proyecto se basará en la predicción del precio por lo que prepararemos los datos para un  modelo de regresión.


In [6]:
#Testing , BORRAR

df = df.sample(1000)

Se renombran las columnas para mantener formato snake_case en el código.

In [5]:
cols = df.columns
snake_case_cols = []

for col in cols:
    snake_case_col = '_'.join(re.findall(r'[A-Z][a-z0-9]*', col)).lower()
    snake_case_cols.append(snake_case_col)

# print(snake_case_cols)
df.columns = snake_case_cols

Obteniendo información general

In [8]:
df.sample(5)

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
157684,22/03/2016 10:41,12799,sedan,2006,manual,265,1er,150000,12,petrol,bmw,no,22/03/2016 00:00,0,59423,05/04/2016 21:48
248973,12/03/2016 10:54,150,NaN,1995,NaN,0,NaN,150000,11,petrol,renault,NaN,12/03/2016 00:00,0,30539,06/04/2016 02:46
214864,24/03/2016 20:57,4000,wagon,2002,manual,184,3er,150000,6,gasoline,bmw,no,24/03/2016 00:00,0,84434,05/04/2016 13:47
94051,26/03/2016 17:38,4800,wagon,2000,auto,193,5er,150000,3,petrol,bmw,no,26/03/2016 00:00,0,63814,26/03/2016 17:38
112545,09/03/2016 11:49,530,coupe,2001,manual,125,other,150000,5,gasoline,saab,no,09/03/2016 00:00,0,85410,31/03/2016 01:18


Revisando la estructura del dataframe

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 89381 to 194401
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_crawled        1000 non-null   object
 1   price               1000 non-null   int64 
 2   vehicle_type        888 non-null    object
 3   registration_year   1000 non-null   int64 
 4   gearbox             946 non-null    object
 5   power               1000 non-null   int64 
 6   model               939 non-null    object
 7   mileage             1000 non-null   int64 
 8   registration_month  1000 non-null   int64 
 9   fuel_type           911 non-null    object
 10  brand               1000 non-null   object
 11  not_repaired        802 non-null    object
 12  date_created        1000 non-null   object
 13  number_of_pictures  1000 non-null   int64 
 14  postal_code         1000 non-null   int64 
 15  last_seen           1000 non-null   object
dtypes: int64(7), object(9)


Identificando columnas con valores nulos

In [10]:
null_values = df.isna().sum()
null_rows= df.shape[0] - df.dropna().shape[0]

null_values = null_values[null_values>0]


print(f'Null values by column:')
print(null_values, end='\n\n')
print(f'Total null rows: {null_rows}')

Null values by column:
vehicle_type    112
gearbox          54
model            61
fuel_type        89
not_repaired    198
dtype: int64

Total null rows: 304


Identificacndo valores duplicados

In [11]:
duplicated_rows = df.duplicated()
duplicated_rows.sum()

0

Examinando manualmente un caso para verificar coherencia

In [12]:
df[(df['date_crawled'] == '21/03/2016 19:06') & (df['price']== 5999)]
# df.query("date_crawled == '21/03/2016 19:06' and price == 5999")

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen


Observaciones:

- Elementos con clasificaciones Dtype erroneos (price, not_repaired, date_crawled, last_seen)
- Elementos nulos para las columnas (vehicle_type, gearbox, model, fuel_type, not_repaired)
- 262 elementos duplicados.

## 2. Limpieza de datos

Se reasignaran los tipos de valor date_crawled y last_seen al formato datetime64, se evitará reasignar los tipos de valor de price y not_repaired debido a sus valores nulos

In [13]:
df['date_crawled'] = pd.to_datetime(df['date_crawled'], format='%d/%m/%Y %H:%M')
df['last_seen'] = pd.to_datetime(df['last_seen'], format='%d/%m/%Y %H:%M')

Verificando cambios realizados

In [14]:
print(df['date_crawled'].dt.year.unique() == df['last_seen'].dt.year.unique())
# print(df['date_crawled'].dt.month.sort_values().unique() == df['last_seen'].dt.month.sort_values().unique())

[ True]


Modificamos la columna 'not_repaired' a booleano

In [15]:
pd.set_option('future.no_silent_downcasting', True)
df['not_repaired'].replace({'yes':1, 'no':0}, inplace=True)

C:\Users\jm_he\AppData\Local\Temp\ipykernel_6928\2027442455.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['not_repaired'].replace({'yes':1, 'no':0}, inplace=True)


Los datos pertenecen al 2016 y las columnas date_crawled y last seen tiene los mismo resultados a nivel año y mes

Procedemos a eliminar los valores duplicados

In [16]:
df = df.drop_duplicates().reset_index(drop=True)

Crearé una función que me permita analizar de una manera visuallos elementos únicos que se encuentran dentro de mis columnas, para verificar si existen datos erroneos o extraños.



In [17]:
def analyze(db):
    obj = db.select_dtypes(include='object').columns
    int = db.select_dtypes(include='int').columns
    
    print('Analisis de elementos Object')
    for i in obj:

        print(f'Elementos de la columna {i}')
        print(db[i].sort_values().unique(), end='\n\n')

    for i in int:

        print(f'Elementos de la columna {i}')
        print(db[i].sort_values().unique(), end='\n\n')



In [18]:
analyze(df)

Analisis de elementos Object
Elementos de la columna vehicle_type
['bus' 'convertible' 'coupe' 'other' 'sedan' 'small' 'suv' 'wagon' nan]

Elementos de la columna gearbox
['auto' 'manual' nan]

Elementos de la columna model
['147' '156' '159' '1_reihe' '1er' '2_reihe' '3_reihe' '3er' '4_reihe'
 '500' '5_reihe' '5er' '601' '6_reihe' '7er' '80' '850' '900' 'a1' 'a3'
 'a4' 'a5' 'a6' 'a8' 'a_klasse' 'agila' 'alhambra' 'almera' 'altea'
 'arosa' 'astra' 'auris' 'avensis' 'aygo' 'beetle' 'berlingo' 'bora'
 'bravo' 'c4' 'c_klasse' 'c_max' 'c_reihe' 'caddy' 'calibra' 'cherokee'
 'citigo' 'civic' 'cl' 'clio' 'clk' 'clubman' 'colt' 'combo' 'cooper'
 'corolla' 'corsa' 'cr_reihe' 'crossfire' 'cuore' 'discovery' 'doblo'
 'e_klasse' 'eos' 'escort' 'fabia' 'fiesta' 'focus' 'fortwo' 'fox'
 'freelander' 'fusion' 'g_klasse' 'galant' 'galaxy' 'glk' 'golf' 'grand'
 'i_reihe' 'ibiza' 'insignia' 'jazz' 'jimny' 'ka' 'kadett' 'kaefer'
 'kangoo' 'laguna' 'leon' 'logan' 'lupo' 'm_klasse' 'm_reihe' 'matiz'
 'mega

In [19]:
def chart_prev(df):
    fig, axes =plt.subplots(ncols=4,nrows=4,figsize=[12,12])

    for ax, column in zip(axes.flatten(), df.columns):
        ax.hist(df[column].dropna(), bins=10, color='g')
        ax.grid(True)
        ax.set_title(column)

    plt.tight_layout()
    plt.show()
    

In [20]:
# chart_prev(df)


### Observaciones


Al momento nos encontramos con las siguientes situaciones:

Elementos nulos para las columnas:
- vehicle_type
- gearbox 
- model
- fuel_type
- not_repaired

A su vez, hemos identificado incongruencias en los datos registrados para las siguientes columnas: 

- `price`: En los valores de precio puedo observar valores de muy bajos que pueden ser considerados `sesgos de información.`
- `registration year` : se pueden observar ver valores superiores al año actual y valores por debajo de 1886 (cuando se creó el primer vehículo), existen muchos `datos con sesgo`.
- `power` la medición esta en caballos de vapor. El primer vehiculo creado tenía una fuerza de CV de 0.75, por lo que es incongruente ver valores de 0 en la base de datos, en su contraparte, el vehículo con mayor velocidad de CV regitrada para 2020 fue de 1800 CV, por lo que `tenemos incongruencias` en los datos cuyos valores superan los a 10,000 CV. 

De un total de 16 columnas, 8 columnas se encuentran sesgadas

### Sesgo de 'price'

`- Existen precios de vehículos por debajo de los 600 dolares.`

Para poder ejecutar un análisis mas profundo es necesario identificar que pudo haber pasado con los datos, una suposición sería que la base se descargo de una plataforma en línea donde los usuarios pueden decidir el precio final de su vehículo, algunos usuarios podrían optar por colocar precios irreales para llamar la atención de los compradores.

Con el fin de alimentar nuestro modelo de una mejor manera, será necesario reclasificar o excluir los precios menores a 600 dolares. Este umbral se ha obtenido de la busqueda de los precios minimos y maximos para la venta de vehiculos en europa de los sitios
- www.coches.com,
- www.milanuncios.com

Si realizaramos un filtro de precios encontrariamos 46,313 filas afectadas que representan el 13.07% del total del dataset

### Sesgo de 'registration_year'
`- Se pueden observar ver valores superiores al año actual y valores por debajo de 1886 (antes del primer automovil creado).`

Siguiendo las fuentes antes mencionadas, podemos ver vehículos ofertados con fecha de registro desde 1980, mantendremos este umbral en nuestro dataset, por otra parte el registro muestra vehiculos con fecha de registro mayor a 2016, fecha en la que se descargaron estos datos. Por lo que datos posteriores no deberían de considerarse para este estudio.

Considerando la depuración del umbral mínimo y máximo, sería necesario depurar 17,978 filas representando un 5.07% del total del dataset.

### Sesgo de 'power'
`- Valores incongruentes en los CV, con valores inferiores a 60 y superiores a 500 en nuestra base de datos.`

Para la seccion de power tenemos como entrada que el primer vehiculo construido tenía una potencia de Caballos de Vapor (CV) de 0.75. 
De acuerdo con las paginas antes mencionadas, los vehiculos pueden tener una potencia entre 60 y 400 cv


Se obtuvieron muestras al azar los vehiculos de la base de datos con registro mayor a 400cv y ningúno clasifico con mas de 250 cv reales, por lo que limitaremos la búsqueda a 300 cv
Si manejamos los umbrales de 60,300 tendríamos un total de 67,268  datos por de purar representando un 18.99% del dataset.


### Sesgo de 'mileage'
`- Existen vehiculos con registro de salida del año en curso y con 150,000 km registrados (datos incongruentes).`

En promedio, los vehiculos generarn un kilometraje de 15,000 a 27,000 kms por año, para minimizar los datos afectados, se excluiran los autos con un kilometraje mayor a 150,000 para añós posteriores a 2011. (27 kms por añó).
19102 filas que corresponden a un 5.39 del dataset


### Primera conclusión

El dataset descargado presenta bastantes datos sesgados e incongruentes, si bien hemos establecido la hipótesis de que la base se descargo de una plataforma en línea donde los usuarios pueden decidir el precio final de su vehículo no podemos asegurar que sea correcta, sin embargo el enfoque principal del de este proyecto se basa en entrenar un modelo de predicción, por lo que será necesario excluir estos datos para tener una base de mejor calidad.

In [21]:
# Excluyendo valores price
recalibrate_df= df.copy()

In [22]:
recalibrate_df= recalibrate_df.query('price>=600')
recalibrate_df = recalibrate_df.query(' 1980 < registration_year < 2017')
recalibrate_df = recalibrate_df.query('60 <=power <=300')
recalibrate_df = recalibrate_df.query('~(registration_year>=2012 and mileage >= 150000)')

In [23]:
loss_data = (1 - (recalibrate_df.shape[0] / df.shape[0])) *100
print(f'Se ha depurado un {loss_data:.2f} de la data original')

Se ha depurado un 30.90 de la data original


In [24]:
recalibrate_df.isna().sum().sort_values(ascending=False)

not_repaired          75
model                 22
fuel_type             15
vehicle_type          11
gearbox               11
date_crawled           0
price                  0
registration_year      0
power                  0
mileage                0
registration_month     0
brand                  0
date_created           0
number_of_pictures     0
postal_code            0
last_seen              0
dtype: int64

In [25]:
# chart_prev(recalibrate_df)

Las columnas de power y de registration year ahora tienen coherencia, el kilometraje sigue estando alto para autos con valores de recorrido mayor a 150,000, de acuerdo con la información externa, esta puede entrar en el umbral de 27,000 kms por año.

### Valores nuloss

Para la manipulación de los valores nulos, se han analizado las siguientes opciones:
- Eliminación de filas o columnas con valores nulos
- Imputación de valores nulos
`- Uso de modelos de imputación`
- Codificación de valores nulos

Optaré por el uso de modelos de imputación

In [26]:
imputated_df = recalibrate_df.dropna()


Mantenemos las columnas que realmente nos interesan

In [27]:
imputated_df = imputated_df.drop(columns=['date_crawled','date_created','number_of_pictures', 'last_seen','postal_code'])

# chart_prev(imputated_df)


In [28]:
imputated_df.sample(5)

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
319,10990,sedan,2009,manual,143,1er,125000,1,gasoline,bmw,0
152,10990,convertible,2012,manual,120,2_reihe,30000,2,petrol,peugeot,0
560,19900,sedan,2009,auto,235,x_reihe,150000,12,gasoline,bmw,0
512,6300,wagon,2008,manual,140,passat,150000,4,gasoline,volkswagen,0
949,2700,small,2006,manual,75,fusion,150000,3,petrol,ford,0


## Entrenamiento del modelo 

In [29]:
def OHE(df):
    encoder = OneHotEncoder(drop='first')
    encoded_data = encoder.fit_transform(df)
    output_names = encoder.get_feature_names_out(df.columns)
    df_codified = pd.DataFrame(encoded_data.toarray(), columns=output_names, index= df.index)
    return df_codified

Standarizando valores numéricos para mejorar los resultados de el modelo LinerarRegression, los modelos RandomForest y LightGBM no se verán beneficiados ni perjudicados por esta manipulación.

In [30]:
obj_cols = imputated_df.select_dtypes(include=['object']).columns
new_cols = OHE(imputated_df[obj_cols])

imputated_df =imputated_df.drop(columns=obj_cols)
imputated_df = pd.concat([imputated_df, new_cols], axis=1)

In [31]:
X = imputated_df.drop(columns=['price'])
y = imputated_df['price']

In [32]:
X.head()

,registration_year,power,mileage,registration_month,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,...,brand_renault,brand_saab,brand_seat,brand_skoda,brand_smart,brand_suzuki,brand_toyota,brand_volkswagen,brand_volvo,not_repaired_1
0,1996,193,150000,4,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011,170,125000,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1998,150,150000,4,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,2007,200,150000,6,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2011,69,40000,12,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
y.head()

0     1650
3    15950
4     1050
6     5750
8     6550
Name: price, dtype: int64

In [34]:
def scaler(X_train, X_test):
    int_names = X_train.select_dtypes(include=['int']).columns
    model =  StandardScaler()
    X_train[int_names] = model.fit_transform(X_train[int_names])
    X_test[int_names] = model.transform(X_test[int_names])
    return X_train, X_test

In [35]:
def model_training(X,y, model, params):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=100) 

    # Código prueba

    X_train, X_test = scaler(X_train, X_test)

    # print(X_train)
    # print(X_test)

    grid_search = GridSearchCV(model, params, scoring='neg_root_mean_squared_error', cv=5, verbose=3)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)

    rmse = root_mean_squared_error(y_test,y_pred)

    return rmse

In [36]:
times = []

### Linear Regression Model 

In [37]:
start_time = time.time()

model = LinearRegression()
params = {
    'fit_intercept': [True, False],
}
rmse_lr = model_training(X,y,model,params)

end_time = time.time()
elapsed_time = end_time - start_time


times.append({'Modelo':'Linear Regression','RMSE':rmse_lr,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (Linear Regression):", rmse_lr)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END fit_intercept=True;, score=-21646960942688588.000 total time=   1.4s
[CV 2/5] END .fit_intercept=True;, score=-431434998833216.250 total time=   1.4s
[CV 3/5] END .fit_intercept=True;, score=-165731254709948.500 total time=   1.4s
[CV 4/5] END fit_intercept=True;, score=-1649963732600994.750 total time=   1.3s
[CV 5/5] END fit_intercept=True;, score=-2451044527764619.500 total time=   1.1s
[CV 1/5] END fit_intercept=False;, score=-7310749779419101.000 total time=   1.2s
[CV 2/5] END fit_intercept=False;, score=-599932923801641.375 total time=   0.9s
[CV 3/5] END fit_intercept=False;, score=-199346283822690.500 total time=   1.1s
[CV 4/5] END fit_intercept=False;, score=-907180039303652.125 total time=   1.0s
[CV 5/5] END fit_intercept=False;, score=-2380685546118472.000 total time=   1.0s


Tiempo de ejecución: 13.97 segundos
RMSE en el conjunto de entrenamiento (Linear Regression): 4805841094873057.0


### Bosque Aleatorio (Regresión)

In [38]:
start_time = time.time()

model = RandomForestRegressor(random_state=100)
params = {
    'n_estimators': [10],
    # 'n_estimators': [10,20,100],
    # 'max_depth': [None, 10,20]
    # 'min_samples_split': [2,10],
    # 'min_samples_leaf': [1,4]
}
rmse_rf = model_training(X,y,model,params)

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'Random_Forest','RMSE':rmse_rf,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (Random Forest):", rmse_rf)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ...............n_estimators=10;, score=-2529.703 total time=   0.0s
[CV 2/5] END ...............n_estimators=10;, score=-2551.131 total time=   0.0s
[CV 3/5] END ...............n_estimators=10;, score=-1903.262 total time=   0.0s
[CV 4/5] END ...............n_estimators=10;, score=-2843.043 total time=   0.0s
[CV 5/5] END ...............n_estimators=10;, score=-2616.435 total time=   0.0s


Tiempo de ejecución: 0.26 segundos
RMSE en el conjunto de entrenamiento (Random Forest): 2248.9204640668445


### LightGBM Regressor

In [39]:
start_time = time.time()

y= imputated_df['price']
model = LGBMRegressor(objective='regression', metric='rmse', random_state=100)
params = {

    'boosting_type': ['gbdt','dart'],
    # 'num_leaves': [20,30,40],
    'learning_rate': [0.05,0.1,0.2],
    'n_estimators': [50,100]

    # 'boosting_type': ['gbdt','dart'],
    # 'num_leaves': [20,30,40],
    # 'learning_rate': [0.05,0.1,0.2],
    # 'n_estimators': [50,100,200]
}
rmse_lgbm = model_training(X,y,model,params)

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'Light_GBM','RMSE':rmse_lgbm,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (Lightgbm):", rmse_lgbm)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 133
[LightGBM] [Info] Number of data points in the train set: 348, number of used features: 19
[LightGBM] [Info] Start training from score 5985.129310
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV 2/5] END boosting_type=gbdt, learning_rate=0.05, n_estimators=50;, score=-2554.534 total time=   0.0s
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000200 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 140
[LightGBM] [Info] Number of data points in the train set: 348, number of used features: 22
[LightGBM] [Info] Start training from score 6225.482759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

### Catboost Regressor

In [40]:
start_time = time.time()

# Inicializar el modelo de CatBoost
catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=6, loss_function='RMSE', verbose=0)

# Entrenar el modelo de CatBoost
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=100) 
X_train, X_test = scaler(X_train, X_test)

catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Predecir sobre el conjunto de entrenamiento y prueba
y_train_pred_catboost = catboost_model.predict(X_train)
y_test_pred_catboost = catboost_model.predict(X_test)

# Calcular el error cuadrático medio (RMSE) en el conjunto de entrenamiento y prueba
rmse_train_catboost = root_mean_squared_error(y_train, y_train_pred_catboost)
rmse_test_catboost = root_mean_squared_error(y_test, y_test_pred_catboost)

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'Catboost','RMSE':rmse_test_catboost,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (CatBoost):", rmse_train_catboost)
print("RMSE en el conjunto de prueba (CatBoost):", rmse_test_catboost)




Tiempo de ejecución: 2.36 segundos
RMSE en el conjunto de entrenamiento (CatBoost): 841.6423742109038
RMSE en el conjunto de prueba (CatBoost): 1810.320155264897


### XGboost

In [41]:
start_time = time.time()

# Convertir los conjuntos de datos a formato DMatrix de XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definir los parámetros del modelo
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Entrenar el modelo de XGBoost
xgb_model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dtrain, 'train'), (dtest, 'test')], early_stopping_rounds=100, verbose_eval=100)

# Predecir sobre el conjunto de entrenamiento y prueba
y_train_pred_xgb = xgb_model.predict(dtrain)
y_test_pred_xgb = xgb_model.predict(dtest)

# Calcular el error cuadrático medio (RMSE) en el conjunto de entrenamiento y prueba
rmse_train_xgb = root_mean_squared_error(y_train, y_train_pred_xgb )
rmse_test_xgb = root_mean_squared_error(y_test, y_test_pred_xgb)

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'XGBoost','RMSE':rmse_test_xgb,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (XGBoost):", rmse_train_xgb)
print("RMSE en el conjunto de prueba (XGBoost):", rmse_test_xgb)


[0]	train-rmse:4447.30534	test-rmse:4103.91823
[100]	train-rmse:529.64596	test-rmse:1920.44183
[185]	train-rmse:312.49683	test-rmse:1924.45777


Tiempo de ejecución: 0.62 segundos
RMSE en el conjunto de entrenamiento (XGBoost): 312.4968325122262
RMSE en el conjunto de prueba (XGBoost): 1924.4577707496333


## Análisis del modelo

In [48]:
results = pd.DataFrame(times).sort_values(by='RMSE').reset_index(drop=True)
results

,Modelo,RMSE,Tiempo
0,Catboost,1.810320e+03,2.359612
1,XGBoost,1.924458e+03,0.619998
2,Light_GBM,1.964357e+03,5.382351
3,Random_Forest,2.248920e+03,0.258659
4,Linear Regression,4.805841e+15,13.974095


# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores
- [x]  Las celdas con el código han sido colocadas en orden de ejecución
- [x]  Los datos han sido descargados y preparados- [ ]
- [x] Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos